In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING messages
import argparse
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
from sentence_transformers import SentenceTransformer

## train test split
from sklearn.model_selection import train_test_split


/home/dimitri/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Parameters
n_missing = 3
MODEL_PATH = "rnn_model.h5"
TOKENIZER_PATH = "tokenizer.pkl"
LABEL_ENCODER_PATH = "label_encoder.pkl"
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 128

In [3]:
df = pd.read_parquet("hf://datasets/data-is-better-together/10k_prompts_ranked/data/train-00000-of-00001.parquet")

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/datasets/data-is-better-together/10k_prompts_ranked (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7d39247770d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))"), '(Request ID: e60aad07-eacb-4184-86fd-c53e5265d092)')

In [ ]:
df.head()

,prompt,quality,metadata,avg_rating,num_responses,agreement_ratio,raw_responses,kind,cluster_description,topic
0,Provide step-by-step instructions on how to ma...,[{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb39...,"{""source"": ""ultrachat"", ""kind"": ""synthetic"", ""...",5.000000,2,1.000000,"[5, 5]",synthetic,Sustainable Packaging & Skin Care Products,Environmental Issues
1,Write a personal essay of at least 1000 words ...,[{'user_id': '6621c3f0-1af2-4d75-acda-ed9c78b9...,"{""source"": ""ultrachat"", ""kind"": ""synthetic"", ""...",2.750000,4,0.687500,"[2, 3, 3, 3]",synthetic,Educational Technology & Cybersecurity in Fash...,Science and Technology
2,"In this research, we aim to investigate how te...",[{'user_id': '2e6dda25-0a99-45aa-a02d-65f426d6...,"{""source"": ""evol_instruct"", ""kind"": ""synthetic...",3.000000,3,0.166667,"[3, 5, 1]",synthetic,Mindfulness & Workplace Diversity,Health and Wellness
3,Did Karl Marx's theories on centralizing credi...,[{'user_id': 'd23b12c2-b601-490e-b5b3-2040eb39...,"{""source"": ""OpenAssistant/oasst2"", ""kind"": ""hu...",3.500000,2,0.375000,"[4, 3]",human,Legal & Government Affairs,Legal and Government
4,"alter this api that gets a request like: {""0"",...",[{'user_id': '99a4bc7d-3e95-4c18-a8f1-26043abf...,"{""source"": ""ewof/sharegpt-instruct-unfiltered-...",3.666667,3,0.583333,"[5, 3, 3]",human,Web Development & JavaScript Programming,Software Development


In [ ]:
data = df.prompt
note = df.avg_rating

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM

# Charger le tokenizer et le modèle BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
bert_model = AutoModel.from_pretrained("bert-base-multilingual-cased")

In [4]:
from tqdm import tqdm

In [ ]:
# Exemple de données
texts = list(data)[:5000]

batch_size = 528  # Ajustez cette valeur en fonction de votre RAM

all_bert_embeddings = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i : i + batch_size]
    encoded_inputs = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=MAX_SEQUENCE_LENGTH,
        return_tensors="pt"
    )
    
    with torch.no_grad():
        outputs = bert_model(**encoded_inputs)
        batch_embeddings = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)
    
    all_bert_embeddings.append(batch_embeddings.numpy())

# Concaténer tous les embeddings en un seul tableau numpy
bert_embeddings = np.concatenate(all_bert_embeddings, axis=0)

# Entraîner le modèle avec des données d'entraînement adaptées
# model.fit(bert_embeddings_train, y_train, validation_data=(bert_embeddings_test, y_test), epochs=10, batch_size=32)

 75%|███████▌  | 15/20 [09:40<03:16, 39.40s/it]

: 

In [52]:
bert_embeddings.shape

(500, 68, 768)

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings, np.array(list(note)), test_size=0.2, random_state=42)

In [9]:
# Construire le modèle LSTM avec les embeddings de BERT
model = Sequential()

# Ajouter des couches bidirectionnelles LSTM
model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(MAX_SEQUENCE_LENGTH, bert_embeddings.shape[-1]))))
model.add(Bidirectional(LSTM(64)))

# Ajouter des couches denses
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compiler le modèle
model.compile(loss='mean_absolute_error', optimizer='adam')

In [11]:
# Train and evaluate model with train and test
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
25/25 [==============================] - 8s 213ms/step - loss: 1.0481 - val_loss: 0.7871
Epoch 2/10
25/25 [==============================] - 4s 162ms/step - loss: 0.7149 - val_loss: 0.7131
Epoch 3/10
25/25 [==============================] - 4s 159ms/step - loss: 0.6701 - val_loss: 0.7030
Epoch 4/10
25/25 [==============================] - 5s 188ms/step - loss: 0.6561 - val_loss: 0.8430
Epoch 5/10
25/25 [==============================] - 4s 156ms/step - loss: 0.5903 - val_loss: 0.7069
Epoch 6/10
25/25 [==============================] - 4s 151ms/step - loss: 0.5813 - val_loss: 0.7183
Epoch 7/10
25/25 [==============================] - 4s 178ms/step - loss: 0.5157 - val_loss: 0.7265
Epoch 8/10
25/25 [==============================] - 4s 154ms/step - loss: 0.4125 - val_loss: 0.7640
Epoch 9/10
25/25 [==============================] - 4s 166ms/step - loss: 0.3674 - val_loss: 0.7579
Epoch 10/10
25/25 [==============================] - 4s 159ms/step - loss: 0.3484 - val_loss: 0.8199

In [63]:
def predict(data):
    data = [data]
    # Tokenisation et encodage des textes
    encoded_inputs = tokenizer(data, padding=True, truncation=True, max_length=MAX_SEQUENCE_LENGTH, return_tensors="pt")
    # Obtenir les embeddings de BERT
    with torch.no_grad():
        outputs = bert_model(**encoded_inputs)
        # Utiliser les embeddings de la dernière couche cachée
        bert_embeddings = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)

    # Convertir les embeddings en numpy pour Keras
    bert_embeddings = bert_embeddings.numpy()
    print(bert_embeddings.shape)
    prediction = model.predict(bert_embeddings)
    return prediction

In [69]:
predict("Un éléphant pèse 5 000 kg et tient sur une patte. Quel est le poids supporté par cette patte ?")

(1, 30, 768)
1/1 [==============================] - 0s 15ms/step


array([[1.9818515]], dtype=float32)